In [1]:
import pandas as pd
import numpy as np
# Read the csv file
df = pd.read_csv('data.csv', header=None)

# Remove empty rows
df = df.replace(np.nan,0)

# Convert the dataframe to a numpy array
data = df.to_numpy()

In [2]:
data

array([[0.0, 'R', 'P', ..., 'R', 'R', 'R']], dtype=object)

In [13]:
data.shape

(1, 200000)

In [9]:
Mapping = {'R':1,'P':2,'S':3}

def map_values(x):
    return Mapping.get(x, x)

In [14]:
vectorized_map_values = np.vectorize(map_values)
converted_data = vectorized_map_values(data)

NameError: name 'v' is not defined

In [11]:
converted_data

array(<generator object <genexpr> at 0x0000021E5C74EFF0>, dtype=object)